# Import library

In [300]:
# #pip install nltk
# import nltk

# # # Download NLTK data
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

In [301]:
#pip install langdetect

In [302]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import string 
import re 
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import spacy
import glob
from langdetect import detect
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem import RegexpStemmer
from sklearn.feature_extraction.text import CountVectorizer

# Preprocessing

In [303]:
# Import dataframe
df = pd.read_csv('assignment_data.csv')

In [304]:
df.head()

,Unnamed: 0,Hotel_id,num_helpful_votes,id,via_mobile,revisit,Rating_overall,Rating_service,Rating_cleanliness,Rating_value,...,Rating_check_in_front_desk,Rating_business_service_(e_g_internet_access),author_id,Author_username,Author_num_cities,Author_num_helpful_votes,Author_num_reviews,Author_location,title,text
0,1,93437,1,89843482,False,Trigger_revisit,4,4.0,4.0,5.0,...,NaN,NaN,0081B56EE5B085710FDCC638FBCF526F,vicbend,NaN,2.0,5,USA,ALWAYS PLEASANT EXPERIENCE!,I have stayed at the Edison on multiple occasi...
1,2,93437,1,127097036,False,No_revisit,4,4.0,3.0,4.0,...,NaN,NaN,0081B56EE5B085710FDCC638FBCF526F,vicbend,NaN,2.0,5,USA,Never a disappointment!,I stayed at the Edison in NYC from March 27-31...
2,3,93437,0,96082073,False,Trigger_revisit,5,4.0,4.0,4.0,...,NaN,NaN,0154B1F9641B89A58EA7BD9FC192EF52,FiveTenLover,14.0,10.0,20,USA,An Art Deco Gem,I visited Edison during a long weekend birthda...
3,4,93437,1,116074323,False,No_revisit,4,4.0,4.0,3.0,...,NaN,NaN,0154B1F9641B89A58EA7BD9FC192EF52,FiveTenLover,14.0,10.0,20,USA,Another Winning Stay At Edison,This was my second stay at Edison. We took a t...
4,5,93437,0,59627462,False,Trigger_revisit,5,5.0,5.0,5.0,...,NaN,NaN,0F81A36F58EA15EB003EACA155D39D7A,marilisag,NaN,NaN,2,USA,Excellent place to stay in NYC,This is my second stay at the Hotel Edison and...


In [305]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3118 entries, 0 to 3117
Data columns (total 23 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Unnamed: 0                                     3118 non-null   int64  
 1   Hotel_id                                       3118 non-null   int64  
 2   num_helpful_votes                              3118 non-null   int64  
 3   id                                             3118 non-null   int64  
 4   via_mobile                                     3118 non-null   bool   
 5   revisit                                        3118 non-null   object 
 6   Rating_overall                                 3118 non-null   int64  
 7   Rating_service                                 2838 non-null   float64
 8   Rating_cleanliness                             2856 non-null   float64
 9   Rating_value                                   2843 

In [306]:
# Concatinate title and review for processing
df['review_concat'] = df['title'] + " " + df['text']

In [307]:
# Filter out only review in English
def filter_english_reviews(df, text_column):
    english_reviews = []
    for index, row in df.iterrows():
        text = row[text_column]
        try:
            if detect(text) == 'en':
                english_reviews.append(index)
        except:
            # Handle cases where language detection fails
            pass
    return df.loc[english_reviews].reset_index()


In [308]:
df_clean = filter_english_reviews(df, 'review_concat')
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Data columns (total 25 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   index                                          2613 non-null   int64  
 1   Unnamed: 0                                     2613 non-null   int64  
 2   Hotel_id                                       2613 non-null   int64  
 3   num_helpful_votes                              2613 non-null   int64  
 4   id                                             2613 non-null   int64  
 5   via_mobile                                     2613 non-null   bool   
 6   revisit                                        2613 non-null   object 
 7   Rating_overall                                 2613 non-null   int64  
 8   Rating_service                                 2376 non-null   float64
 9   Rating_cleanliness                             2389 

## String2WordVector

In [309]:
# Extract review for NLP processing
review = df_clean['review_concat']

### Ver 1

In [310]:
# # Stop words removal
# def remove_stops(text, stops):
#     lemmatizer = WordNetLemmatizer()
#     sno = SnowballStemmer(language='english')
#     Porter = PorterStemmer()
#     regexp = RegexpStemmer('ing$|s$|ed$|able$', min=4)
#     #spacy_lem = spacy.load("en_core_web_sm")
#     # Lowercase the text
#     text = text.lower()
    
#     # Remove URLs
#     text = re.sub(r'http\S+|www.\S+|\w+\.com\b|\w+\.html\b|\w+\.\w+\.com\b', '', text)

#     # Remove numbers
#     text = re.sub(r'[0-9]+', '', text)

#     # Define the words to remove
#     words_to_remove = ['times square', 'time square', 
#                        'square','york','new york', 'new yorker'
#                        'new york city', 'nyc', "i'm", "i've", 'ny'
#                        'yorker', 'manhattan', 'palmer', 'central park','grand central',
#                        'madison garden', 'madison', 'affinia',
#                        'crown plaza', 'marriott', 'midtown', 'park central', 'plaza', 'macys','macy',
#                        'hotel', 'hotels', 'place', 'able','king']

#     # Create a regular expression pattern to match any of the words
#     pattern = r'\b(?:' + '|'.join(map(re.escape, words_to_remove)) + r')\b'

#     # Remove the words using regular expression substitution
#     text = re.sub(pattern, '', text, flags=re.IGNORECASE)

#     # Remove non-word characters and specific abbreviations
#     text = re.sub(r'(?:\s|^)(lul|lol)(?=\s|$)', '', text)
    
#     # Split the text into words
#     words = text.split()
    
#     # Initialize an empty list to store the final words after processing
#     final = []
    
#     # Iterate through each word in the text
#     for word in words:
#         # Check if the word is not in the list of stopwords
#         if word not in stops:
#             # Lemmatize the word and append it to the final list
#             #final.append(sno.stem(lemmatizer.lemmatize(word))) #With snowball stemmer 
#             #final.append(Porter.stem(lemmatizer.lemmatize(word))) #With Porter stemmer 
#             final.append(lemmatizer.lemmatize(word)) #Only Lemmatization with nltk
#             #final.append(spacy_lem(word)) #Lemmatization with spacy
#             #final.append(Porter.stem(word)) #Only Porter
#             #final.append(regexp.stem(lemmatizer.lemmatize(word))) #With Lemmatizaion with Regex
            
    
#     # Join the final list of words into a single string
#     final_text = " ".join(final)

#      # Remove non-word characters and specific abbreviations
#     final_text = re.sub(r'[^\w\s]', '', final_text)

#      # Remove extra whitespaces
#     final_text = re.sub(r'\s+', ' ', final_text)
    
#     return final_text

# # Function for cleaning review
# def clean_docs(docs):
#     stops = stopwords.words('english')
#     months = [
#     "january", "jan",
#     "february", "feb",
#     "march", "mar",
#     "april", "apr",
#     "may", "may",
#     "june", "jun",
#     "july", "jul",
#     "august", "aug",
#     "september", "sep",
#     "october", "oct",
#     "november", "nov",
#     "december", "dec",
#     ]

#     other = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten',
#     'st','nd','rd','th', 'would', 'edison','empire','empire state', 'empire state building',
#     'hilton','hyatt','im','ive','state', 'time square','westin', 'penn',
#     'crown plaza', 'marriott', 'marquis', 'park central', 'new yorker', 'yorker',
#     'times square', 'time square', 'square','york','new york city','nyc',"im","ive",'ny'
#     'manhattan', 'palmer','central park','grand central','madison garden','madison','affinia',
#     'park central', 'plaza', 'Macy', 'Macys', 'could','couldnt','wouldnt']
    
#     cities = [
#     'new york city', 'new york','nyc','york','ny','hudson',
#     'los angeles', 'la',
#     'chicago', 'chi',
#     'houston', 'hou',
#     'phoenix', 'phx',
#     'philadelphia', 'phi',
#     'san antonio', 'sat',
#     'san diego', 'sdo',
#     'dallas', 'dal',
#     'san jose', 'sjo',
#     'austin', 'aus',
#     'jacksonville', 'jax',
#     'san francisco', 'sfo',
#     'columbus', 'col',
#     'indianapolis', 'ind',
#     'seattle', 'sea',
#     'denver', 'den',
#     'washington, d.c.', 'wdc',
#     'boston', 'bos',
#     'el paso', 'elp',
#     'detroit', 'det',
#     'nashville', 'nsh',
#     'portland', 'por',
#     'memphis', 'mem',
#     'oklahoma city', 'okc',
#     'las vegas', 'las',
#     'louisville', 'lou',
#     'baltimore', 'bal',
#     'milwaukee', 'mil',
#     'albuquerque', 'abq',
#     'tucson', 'tuc',
#     'fresno', 'fre',
#     'sacramento', 'sac',
#     'mesa', 'mes',
#     'kansas city', 'kan',
#     'atlanta', 'atl',
#     'miami', 'mia',
#     'minneapolis', 'min',
#     'new orleans', 'nol',
#     'tampa', 'tam',
#     'orlando', 'orl',
#     'pittsburgh', 'pit',
#     'raleigh', 'ral',
#     'cleveland', 'cle',
#     'wichita', 'wic',
#     'anaheim', 'ana',
#     'honolulu', 'hon',
#     'santa ana', 'sta',
#     'corpus christi', 'cor',
#     'lexington', 'lex',
#     'henderson', 'hen',
#     'anchorage', 'anc',
#     'st. louis', 'stl',
#     'stockton', 'sto',
#     'reno', 'ren',
#     'scottsdale', 'sco',
#     'buffalo', 'buf',
#     'gilbert', 'gil',
#     'glendale', 'gle',
#     'north las vegas', 'nlv',
#     'chandler', 'cha',
#     'hialeah', 'hia',
#     'garland', 'gar',
#     'irvine', 'irv',
#     'chesapeake', 'che',
#     'irving', 'irv',
#     'fremont', 'fre',
#     'richmond', 'ric',
#     'lubbock', 'lub',
#     'boise', 'boi',
#     'baton rouge', 'bat',
#     'des moines', 'des',
#     'spokane', 'spo',
#     'yakima', 'yak',
#     'olympia', 'oly',
#     'everett', 'eve',
#     'tacoma', 'tac',
#     'bellingham', 'bel',
#     'bremerton', 'bre',
#     'vancouver', 'van',
#     'portland', 'por',
#     'eugene', 'eug',
#     'salem', 'sal',
#     'medford', 'med',
#     'bend', 'ben',
#     'roseburg', 'ros',
#     'grants pass', 'gra',
#     'ashland', 'ash',
#     'klamath falls', 'kla',
#     'redding', 'red',
#     'manhattan'
# ]
#     stops = stops + months + cities + other
#     final = []
#     for doc in docs: 
#         clean_docs = remove_stops(doc, stops)
#         final.append(clean_docs)
#     return (final)


### Ver 2

In [438]:
# Stop words removal
# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

def remove_interjections_and_stopwords(text, stops):
    # # Load the English language model in spaCy
    # nlp = spacy.load("en_core_web_sm")
    
    # Remove numbers
    text = re.sub(r'[0-9]+', '', text)

    # Remove non-word characters 
    #text = re.sub(r'[^\w\s]', '', text)
    
    # Process the text (tokenize)
    doc = nlp(text)
    
    # Initialize an empty list to store non-interjection and non-stopword words
    non_interjections_and_stopwords = []
    
    # Iterate through each token in the document
    for token in doc:
        # Check if the part-of-speech tag of the token is not "INTJ" (interjection)
        # and the lowercase text of the token is not in the list of stopwords
        if (token.pos_ == "ADJ" or token.pos_ == 'NN' or token.pos == 'ADV') and token.lower_ not in stops:
            if token.text in ['bigger', 'biggest', 'smaller', 'smallest', 'older','oldest','larger','largest','nicer','nicest']:
                non_interjections_and_stopwords.append(token.lemma_.lower())
            else:
                non_interjections_and_stopwords.append(token.text.lower())
            # # Append the lowercase text of the token to the list
            # non_interjections_and_stopwords.append(token.lemma_.lower())
    
    # Join the list of non-interjection and non-stopword words into a single string
    final_text = " ".join(non_interjections_and_stopwords)

    # Remove non-word characters 
    final_text = re.sub(r'[^\w\s]', '', final_text)

    # Perform substitution using re.sub()
    final_text = re.sub(r'\bok\b', 'okay', final_text)
    
    return final_text
    
    
# Function for cleaning review
def clean_docs(docs):
    stops = stopwords.words('english')

    other = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten',
             'first','second','third', 'fourth','fifth','sixth','seventh','eighth','nineth','tenth',
            'st','nd','rd','th', 'would', 'edison','empire','empire state', 'empire state building',
            'hilton','hyatt','im','ive','state', 'time square','westin', 'penn',
            'crown plaza', 'marriott', 'marquis', 'park central', 'new yorker', 'yorker',
            'times square', 'time square', 'square','york','new york city','nyc',"im","ive",'ny'
            'manhattan', 'palmer','central park','grand central','madison garden','madison','affinia',
            'park central', 'plaza', 'Macy', 'Macys', 'could','couldnt','wouldnt', 'central','entire',
            'past','next','therecommend','prompt','south','north','east','west', 'able', 'grand','wireless',
            'sure','clear','seperate','single','sized','regular','flat','nearby','young','open','overall','main','non', 'twin','white','personal','double',
            'black','lucky','willing','continental','close','aware','complimentary','executive','general','light','local',
            'available','bedding','bright','certain','complete','dark','different',
            'extra','key','midtown','mini','multiple','opposite','original','particular',
            'separate','TRUE','true','yellow','upper','public','soft', 'favorite',
            'plus','walkable','wrong','minor','major','several','historic','early', 'low', 'lower','high','higher','lowest','highest','additional','hot','cold']
    
    stops = stops + other
     
    final = []
    for doc in docs: 
        clean_docs = remove_interjections_and_stopwords(doc, stops)
        final.append(clean_docs)
    return (final)


In [439]:
#print(review[3])
print(clean_docs(['clean better non main young yellow original,open,overall,test,great,staff,great,dead,fined,good,hi,hello,bigger, smaller,biggest best, best',
                  "The bigger house is bigger than the smaller one, okay, ok, ok","It was okay",'It was ok highly',"I would highly recommend"]))

['better great great dead good big small big best best', 'big big small', 'okay', '', '']


In [440]:
# Perform cleaning
cleaned_docs = clean_docs(review)

In [441]:
clean = pd.DataFrame(cleaned_docs, columns=['review_cleaned'])
clean.to_csv('Preprocessed_file/review_cleaned.csv', index=False, sep =',')

In [442]:
# Checking
for i in range(300,302):
    print("Original:",review[i])
    print("Clean:",cleaned_docs[i])
    print()

Original: Splendor and Elegance The beauty of the lobby is only exceeded by the fabulous attention to excellent service. It is rich in history and yet modern day amenities. Superb on every level. And pet friendly.
Clean: fabulous excellent rich modern friendly

Original: Fabulous historic Chicago hotel This is a must see and stay. Experience the opulence and historic grandeur of the gift Potter Palmer gave to his bride as a wedding gift, The Palmer House. The lobby ceilings were restored by Lido Lippi. The same Lido Lippi who restored the Sistine Chapel.The winged angels outside the the entrance to Empire Room were designed by non other than Louis Comfort Tiffany.
Be sure to visit the Lakewood dining room as have one of Bertha Palmer's famous Chocolate Fudge Brownies. It is worth the trip just for the brownie and people watch in the lobby.
If you have the opportunity take the tour  History is Hott with Ken Price.
This Chicago landmark is one gracious, elegant lady.
Clean: fabulous famo

In [446]:
# Define Tf-IDF vectorization
vectorize = TfidfVectorizer(
    lowercase = True,
    max_features= 200,
    min_df = 30,
    stop_words='english',
    strip_accents = 'unicode'
)

# vectorize words
vectors = vectorize.fit_transform(cleaned_docs)

In [317]:
# # Initialize CountVectorizer
# vectorizer = CountVectorizer( 
#     lowercase = True,
#     max_features= 200,
#     #max_df = 0.7,
#     min_df=50,
#     #ngram_range=(1,3),
#     stop_words='english',
#     strip_accents = 'unicode'
#     )

# # vectorize words
# vectors = vectorizer.fit_transform(cleaned_docs)

In [447]:
# Extract word name
feature_names = vectorize.get_feature_names_out()

In [448]:
feature_names

array(['adequate', 'amazing', 'annoying', 'attentive', 'average',
       'awesome', 'awful', 'bad', 'basic', 'beautiful', 'best', 'better',
       'big', 'brilliant', 'busy', 'cheap', 'cheaper', 'clean',
       'comfortable', 'comfy', 'convenient', 'cool', 'courteous',
       'crowded', 'decent', 'difficult', 'dirty', 'disappointed',
       'disappointing', 'easy', 'efficient', 'enjoyable', 'excellent',
       'exceptional', 'expensive', 'fabulous', 'fair', 'fantastic',
       'fast', 'filthy', 'fine', 'free', 'fresh', 'friendly', 'glad',
       'good', 'gorgeous', 'great', 'handy', 'happy', 'hard', 'helpful',
       'horrible', 'huge', 'ideal', 'important', 'impressed',
       'impressive', 'incredible', 'interesting', 'large', 'late',
       'little', 'long', 'loud', 'lovely', 'massive', 'modern',
       'negative', 'new', 'nice', 'noisy', 'normal', 'numerous', 'odd',
       'okay', 'old', 'outside', 'outstanding', 'perfect', 'pleasant',
       'pleased', 'polite', 'poor', 'positive'

In [449]:
#Create dataframe to join with the main dataframe
final_word2vec = pd.DataFrame(vectors.toarray(), columns = feature_names)

In [450]:
final_word2vec.head()

,adequate,amazing,annoying,attentive,average,awesome,awful,bad,basic,beautiful,...,trendy,typical,uncomfortable,usual,various,warm,wonderful,worse,worst,worth
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.580390,0.0,0.0,0.0
1,0.378717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.331621,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [452]:
final_word2vec.to_csv('Preprocessed_file/word2vec.csv', index = False)

## Main document cleaning and fill null value and export to csv file

In [453]:
# Remove unecessary columns and columns with too much null value
df_clean.drop(columns=['Hotel_id','index','Unnamed: 0','id','author_id','Author_username',
                                  'Rating_check_in_front_desk',
                                  'Rating_business_service_(e_g_internet_access)',
                                  'title',
                                  'text',
                                  'review_concat',
                                  ], inplace = True)


KeyError: "['Hotel_id', 'index', 'Unnamed: 0', 'id', 'author_id', 'Author_username', 'Rating_check_in_front_desk', 'Rating_business_service_(e_g_internet_access)', 'title', 'text', 'review_concat'] not found in axis"

In [454]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num_helpful_votes         2613 non-null   int64  
 1   via_mobile                2613 non-null   bool   
 2   revisit                   2613 non-null   object 
 3   Rating_overall            2613 non-null   int64  
 4   Rating_service            2376 non-null   float64
 5   Rating_cleanliness        2389 non-null   float64
 6   Rating_value              2381 non-null   float64
 7   Rating_location           2147 non-null   float64
 8   Rating_sleep_quality      1553 non-null   float64
 9   Rating_rooms              2235 non-null   float64
 10  Author_num_cities         2060 non-null   float64
 11  Author_num_helpful_votes  2242 non-null   float64
 12  Author_num_reviews        2613 non-null   int64  
 13  Author_location           2613 non-null   object 
dtypes: bool(

### Scenerio 1

#### Rating_overall

In [455]:
rating_overall = pd.concat([df_clean['Rating_overall'],final_word2vec],axis =1)
rating_overall['Rating_overall'] = rating_overall['Rating_overall'].apply(lambda x: 'low' if x < 4 else 'high')
rating_overall.dropna(inplace = True)
rating_overall.reset_index(inplace= True)
rating_overall = rating_overall.iloc[:,1:]
rating_overall.info()
rating_overall.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, Rating_overall to worth
dtypes: float64(123), object(1)
memory usage: 2.5+ MB


Rating_overall    0
adequate          0
amazing           0
annoying          0
attentive         0
                 ..
warm              0
wonderful         0
worse             0
worst             0
worth             0
Length: 124, dtype: int64

In [456]:
rating_overall['Rating_overall'].value_counts()

Rating_overall
high    1765
low      848
Name: count, dtype: int64

In [457]:
rating_overall.to_csv('Preprocessed_file/Scenerio_1_overall.csv', index = False)

#### Rating_service

In [458]:
rating_service = pd.concat([df_clean['Rating_service'],final_word2vec],axis =1)
rating_service['Rating_service'] = rating_service['Rating_service'].apply(lambda x: 'low' if x < 4 else 'high')
rating_service.dropna(inplace = True)
rating_service.reset_index(inplace= True)
rating_service = rating_service.iloc[:,1:]
rating_service.info()
rating_service.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, Rating_service to worth
dtypes: float64(123), object(1)
memory usage: 2.5+ MB


Rating_service    0
adequate          0
amazing           0
annoying          0
attentive         0
                 ..
warm              0
wonderful         0
worse             0
worst             0
worth             0
Length: 124, dtype: int64

In [459]:
rating_service['Rating_service'].value_counts()

Rating_service
high    1828
low      785
Name: count, dtype: int64

In [460]:
rating_service.to_csv('Preprocessed_file/Scenerio_1_service.csv', index = False)

#### Rating_cleanliness

In [461]:
rating_cleanliness = pd.concat([df_clean['Rating_cleanliness'],final_word2vec],axis =1)
rating_cleanliness['Rating_cleanliness'] = rating_cleanliness['Rating_cleanliness'].apply(lambda x: 'low' if x < 4 else 'high')
rating_cleanliness.dropna(inplace = True)
rating_cleanliness.reset_index(inplace= True)
rating_cleanliness = rating_cleanliness.iloc[:,1:]
rating_cleanliness.info()
rating_cleanliness.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, Rating_cleanliness to worth
dtypes: float64(123), object(1)
memory usage: 2.5+ MB


Rating_cleanliness    0
adequate              0
amazing               0
annoying              0
attentive             0
                     ..
warm                  0
wonderful             0
worse                 0
worst                 0
worth                 0
Length: 124, dtype: int64

In [462]:
rating_cleanliness['Rating_cleanliness'].value_counts()

Rating_cleanliness
high    1953
low      660
Name: count, dtype: int64

In [463]:
rating_cleanliness.to_csv('Preprocessed_file/Scenerio_1_cleanliness.csv', index = False)

#### Rating_value

In [464]:
rating_value = pd.concat([df_clean['Rating_value'],final_word2vec],axis =1)
rating_value['Rating_value'] = rating_value['Rating_value'].apply(lambda x: 'low' if x < 4 else 'high')
rating_value.dropna(inplace = True)
rating_value.reset_index(inplace= True)
rating_value = rating_value.iloc[:,1:]
rating_value.info()
rating_value.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, Rating_value to worth
dtypes: float64(123), object(1)
memory usage: 2.5+ MB


Rating_value    0
adequate        0
amazing         0
annoying        0
attentive       0
               ..
warm            0
wonderful       0
worse           0
worst           0
worth           0
Length: 124, dtype: int64

In [465]:
rating_value['Rating_value'].value_counts()

Rating_value
high    1662
low      951
Name: count, dtype: int64

In [466]:
rating_value.to_csv('Preprocessed_file/Scenerio_1_value.csv', index = False)

#### Rating location

In [467]:
rating_location = pd.concat([df_clean['Rating_location'],final_word2vec],axis =1)
rating_location['Rating_location'] = rating_location['Rating_location'].apply(lambda x: 'low' if x < 4 else 'high')
rating_location.dropna(inplace = True)
rating_location.reset_index(inplace= True)
rating_location = rating_location.iloc[:,1:]
rating_location.info()
rating_location.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, Rating_location to worth
dtypes: float64(123), object(1)
memory usage: 2.5+ MB


Rating_location    0
adequate           0
amazing            0
annoying           0
attentive          0
                  ..
warm               0
wonderful          0
worse              0
worst              0
worth              0
Length: 124, dtype: int64

In [468]:
rating_location['Rating_location'].value_counts()

Rating_location
high    2488
low      125
Name: count, dtype: int64

In [469]:
rating_location.to_csv('Preprocessed_file/Scenerio_1_location.csv', index = False)

#### Rating sleep quality

In [470]:
rating_sleep_quality = pd.concat([df_clean['Rating_sleep_quality'],final_word2vec],axis =1)
rating_sleep_quality['Rating_sleep_quality'] = rating_sleep_quality['Rating_sleep_quality'].apply(lambda x: 'low' if x < 4 else 'high')
rating_sleep_quality.dropna(inplace = True)
rating_sleep_quality.reset_index(inplace= True)
rating_sleep_quality = rating_sleep_quality.iloc[:,1:]
rating_sleep_quality.info()
rating_sleep_quality.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, Rating_sleep_quality to worth
dtypes: float64(123), object(1)
memory usage: 2.5+ MB


Rating_sleep_quality    0
adequate                0
amazing                 0
annoying                0
attentive               0
                       ..
warm                    0
wonderful               0
worse                   0
worst                   0
worth                   0
Length: 124, dtype: int64

In [471]:
rating_sleep_quality['Rating_sleep_quality'].value_counts()

Rating_sleep_quality
high    2170
low      443
Name: count, dtype: int64

In [472]:
rating_sleep_quality.to_csv('Preprocessed_file/Scenerio_1_sleep.csv', index = False)

#### Rating rooms

In [473]:
rating_rooms = pd.concat([df_clean['Rating_rooms'],final_word2vec],axis =1)
rating_rooms['Rating_rooms'] = rating_rooms['Rating_rooms'].apply(lambda x: 'low' if x < 4 else 'high')
rating_rooms.dropna(inplace = True)
rating_rooms.reset_index(inplace= True)
rating_rooms = rating_rooms.iloc[:,1:]
rating_rooms.info()
rating_rooms.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, Rating_rooms to worth
dtypes: float64(123), object(1)
memory usage: 2.5+ MB


Rating_rooms    0
adequate        0
amazing         0
annoying        0
attentive       0
               ..
warm            0
wonderful       0
worse           0
worst           0
worth           0
Length: 124, dtype: int64

In [474]:
rating_rooms['Rating_rooms'].value_counts()

Rating_rooms
high    1748
low      865
Name: count, dtype: int64

In [475]:
rating_rooms.to_csv('Preprocessed_file/Scenerio_1_rooms.csv', index = False)

### Scenerio 2

In [476]:
helpful_votes = pd.concat([df_clean['num_helpful_votes'],final_word2vec],axis =1)
helpful_votes.dropna(inplace = True)
helpful_votes.reset_index(inplace= True)
helpful_votes = helpful_votes.iloc[:,1:]
helpful_votes.info()
helpful_votes.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Columns: 124 entries, num_helpful_votes to worth
dtypes: float64(123), int64(1)
memory usage: 2.5 MB


num_helpful_votes    0
adequate             0
amazing              0
annoying             0
attentive            0
                    ..
warm                 0
wonderful            0
worse                0
worst                0
worth                0
Length: 124, dtype: int64

In [477]:
helpful_votes['num_helpful_votes'] = helpful_votes['num_helpful_votes'] >= 15 
helpful_votes[helpful_votes['num_helpful_votes']== True]

,num_helpful_votes,adequate,amazing,annoying,attentive,average,awesome,awful,bad,basic,...,trendy,typical,uncomfortable,usual,various,warm,wonderful,worse,worst,worth
291,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
576,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1048,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1140,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1417,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.559207,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.373971,0.0
1650,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.420325,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1738,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.180263,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1879,True,0.000000,0.0,0.290979,0.0,0.000000,0.0,0.0,0.383112,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.192048,0.0,0.000000,0.0
2102,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
2185,True,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0


In [478]:
helpful_votes.to_csv('Preprocessed_file/Scenerio_2.csv', index = False)

### Scenerio 3

In [479]:
# author_num_cities = pd.concat([df_clean, final_word2vec],axis =1)
# author_num_cities = author_num_cities[author_num_cities['Author_num_cities'].notna()]
# author_num_cities = author_num_cities.reset_index()
# author_num_cities = author_num_cities.iloc[:,1:]
# author_num_cities.info()
# author_num_cities.head()

In [480]:
author_num_cities = df[df['Author_num_cities'].notna()]
author_num_cities.drop(columns = ['title','text','review_concat','id','Rating_check_in_front_desk',
                                  'Rating_business_service_(e_g_internet_access)','author_id','Hotel_id','Author_username'
                                  ],inplace = True)
author_num_cities = author_num_cities.reset_index()
author_num_cities = author_num_cities.iloc[:,2:]
author_num_cities.info()
author_num_cities.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num_helpful_votes         2474 non-null   int64  
 1   via_mobile                2474 non-null   bool   
 2   revisit                   2474 non-null   object 
 3   Rating_overall            2474 non-null   int64  
 4   Rating_service            2258 non-null   float64
 5   Rating_cleanliness        2273 non-null   float64
 6   Rating_value              2261 non-null   float64
 7   Rating_location           2042 non-null   float64
 8   Rating_sleep_quality      1530 non-null   float64
 9   Rating_rooms              2122 non-null   float64
 10  Author_num_cities         2474 non-null   float64
 11  Author_num_helpful_votes  2322 non-null   float64
 12  Author_num_reviews        2474 non-null   int64  
 13  Author_location           2474 non-null   object 
dtypes: bool(

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20776\2292355942.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_num_cities.drop(columns = ['title','text','review_concat','id','Rating_check_in_front_desk',


,num_helpful_votes,via_mobile,revisit,Rating_overall,Rating_service,Rating_cleanliness,Rating_value,Rating_location,Rating_sleep_quality,Rating_rooms,Author_num_cities,Author_num_helpful_votes,Author_num_reviews,Author_location
0,0,False,Trigger_revisit,5,4.0,4.0,4.0,5.0,4.0,4.0,14.0,10.0,20,USA
1,1,False,No_revisit,4,4.0,4.0,3.0,NaN,4.0,NaN,14.0,10.0,20,USA
2,1,False,No_revisit,4,4.0,5.0,5.0,5.0,4.0,3.0,5.0,1.0,9,Austria
3,3,False,Trigger_revisit,4,4.0,4.0,3.0,5.0,3.0,3.0,4.0,3.0,7,UK
4,0,True,No_revisit,4,4.0,4.0,3.0,5.0,NaN,4.0,4.0,3.0,7,UK


In [481]:
rating_to_fill = ['Rating_service','Rating_cleanliness','Rating_value','Rating_location','Rating_sleep_quality','Rating_rooms']
for element in rating_to_fill:
    author_num_cities[element].fillna(3, inplace = True)

author_num_cities['Author_num_helpful_votes'].fillna(0, inplace=True)

author_num_cities['Author_num_cities'] = author_num_cities['Author_num_cities'] >= 15 
author_num_cities[author_num_cities['Author_num_cities']==True]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20776\1222659873.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  author_num_cities[element].fillna(3, inplace = True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20776\1222659873.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

,num_helpful_votes,via_mobile,revisit,Rating_overall,Rating_service,Rating_cleanliness,Rating_value,Rating_location,Rating_sleep_quality,Rating_rooms,Author_num_cities,Author_num_helpful_votes,Author_num_reviews,Author_location
18,1,False,Trigger_revisit,3,3.0,3.0,3.0,5.0,2.0,2.0,True,42.0,67,Canada
19,0,False,No_revisit,2,3.0,3.0,3.0,3.0,3.0,3.0,True,42.0,67,Canada
31,1,False,Trigger_revisit,4,3.0,3.0,3.0,3.0,3.0,3.0,True,28.0,55,USA
32,2,False,Trigger_revisit,4,4.0,4.0,4.0,3.0,4.0,3.0,True,28.0,55,USA
33,1,False,No_revisit,4,3.0,3.0,4.0,5.0,4.0,3.0,True,28.0,55,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,0,False,No_revisit,5,4.0,4.0,4.0,5.0,5.0,5.0,True,30.0,38,Australia
2450,5,False,No_revisit,4,4.0,5.0,4.0,5.0,3.0,4.0,True,20.0,24,UK
2455,6,False,No_revisit,5,4.0,4.0,4.0,5.0,3.0,5.0,True,80.0,38,Germany
2469,1,False,No_revisit,4,4.0,5.0,5.0,5.0,3.0,4.0,True,99.0,109,Australia


In [482]:
author_num_cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2474 entries, 0 to 2473
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num_helpful_votes         2474 non-null   int64  
 1   via_mobile                2474 non-null   bool   
 2   revisit                   2474 non-null   object 
 3   Rating_overall            2474 non-null   int64  
 4   Rating_service            2474 non-null   float64
 5   Rating_cleanliness        2474 non-null   float64
 6   Rating_value              2474 non-null   float64
 7   Rating_location           2474 non-null   float64
 8   Rating_sleep_quality      2474 non-null   float64
 9   Rating_rooms              2474 non-null   float64
 10  Author_num_cities         2474 non-null   bool   
 11  Author_num_helpful_votes  2474 non-null   float64
 12  Author_num_reviews        2474 non-null   int64  
 13  Author_location           2474 non-null   object 
dtypes: bool(

In [483]:
author_num_cities.to_csv('Preprocessed_file/Scenerio_3.csv', index = False)

### Scenerio 4

In [484]:
author_helpful_votes = pd.concat([df_clean, final_word2vec],axis =1)
author_helpful_votes = author_helpful_votes[author_helpful_votes['Author_num_helpful_votes'].notna()]
author_helpful_votes = author_helpful_votes.reset_index()
author_helpful_votes = author_helpful_votes.iloc[:,1:]
author_helpful_votes.info()
author_helpful_votes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Columns: 137 entries, num_helpful_votes to worth
dtypes: bool(1), float64(131), int64(3), object(2)
memory usage: 2.3+ MB


,num_helpful_votes,via_mobile,revisit,Rating_overall,Rating_service,Rating_cleanliness,Rating_value,Rating_location,Rating_sleep_quality,Rating_rooms,...,trendy,typical,uncomfortable,usual,various,warm,wonderful,worse,worst,worth
0,1,False,Trigger_revisit,4,4.0,4.0,5.0,5.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.580390,0.0,0.0,0.0
1,1,False,No_revisit,4,4.0,3.0,4.0,5.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0,False,Trigger_revisit,5,4.0,4.0,4.0,5.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.331621,0.0,0.0,0.0
3,1,False,No_revisit,4,4.0,4.0,3.0,NaN,4.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,3,False,Trigger_revisit,4,4.0,4.0,3.0,5.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [485]:
rating_to_fill = ['Rating_service','Rating_cleanliness','Rating_value','Rating_location','Rating_sleep_quality','Rating_rooms']
for element in rating_to_fill:
    author_helpful_votes[element].fillna(3, inplace = True)

author_helpful_votes['Author_num_cities'].fillna(0, inplace=True)

author_helpful_votes['Author_num_helpful_votes'] = author_helpful_votes['Author_num_helpful_votes'] >= 100
author_helpful_votes[author_helpful_votes['Author_num_helpful_votes']==True]

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20776\4129565120.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  author_helpful_votes[element].fillna(3, inplace = True)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20776\4129565120.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

,num_helpful_votes,via_mobile,revisit,Rating_overall,Rating_service,Rating_cleanliness,Rating_value,Rating_location,Rating_sleep_quality,Rating_rooms,...,trendy,typical,uncomfortable,usual,various,warm,wonderful,worse,worst,worth
44,1,False,Trigger_revisit,4,4.0,4.0,5.0,5.0,4.0,3.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
45,2,False,No_revisit,4,3.0,4.0,5.0,5.0,5.0,4.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
48,1,False,Trigger_revisit,4,4.0,4.0,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
49,2,False,No_revisit,4,4.0,5.0,3.0,5.0,5.0,3.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
52,1,False,No_revisit,1,2.0,5.0,3.0,5.0,3.0,3.0,...,0.0,0.0,0.0,0.366048,0.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2061,6,False,No_revisit,4,4.0,4.0,5.0,3.0,3.0,4.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.350634,0.0,0.0,0.373296
2106,4,False,No_revisit,5,5.0,5.0,5.0,3.0,3.0,5.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2166,4,False,No_revisit,4,4.0,4.0,3.0,3.0,3.0,4.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
2189,0,False,No_revisit,4,3.0,3.0,3.0,5.0,4.0,3.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.314292


In [486]:
author_helpful_votes.iloc[:,:13].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2242 entries, 0 to 2241
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   num_helpful_votes         2242 non-null   int64  
 1   via_mobile                2242 non-null   bool   
 2   revisit                   2242 non-null   object 
 3   Rating_overall            2242 non-null   int64  
 4   Rating_service            2242 non-null   float64
 5   Rating_cleanliness        2242 non-null   float64
 6   Rating_value              2242 non-null   float64
 7   Rating_location           2242 non-null   float64
 8   Rating_sleep_quality      2242 non-null   float64
 9   Rating_rooms              2242 non-null   float64
 10  Author_num_cities         2242 non-null   float64
 11  Author_num_helpful_votes  2242 non-null   bool   
 12  Author_num_reviews        2242 non-null   int64  
dtypes: bool(2), float64(7), int64(3), object(1)
memory usage: 197.2

In [487]:
author_helpful_votes.to_csv('Preprocessed_file/Scenerio_4.csv', index = False)